In [33]:
# Initial imports
import numpy as np
import pandas as pd
from stockstats import StockDataFrame as sdf
import requests
from dotenv import load_dotenv
import os
import pandas as pd
import numpy as np
import alpaca_trade_api as tradeapi
from pathlib import Path
import lib_copy
%matplotlib inline

In [34]:
load_dotenv()

Python-dotenv could not parse statement starting at line 7
Python-dotenv could not parse statement starting at line 8


True

In [35]:


df = lib_copy.fetch_ohlcv("JPM", "2018-01-12", "2019-06-12")

In [36]:
def adding_boll_kelt_ewma_dataframe(dataframe):
    lib_copy.bollinger_band_generator(dataframe)
    lib_copy.keltner_channel(dataframe)
    lib_copy.ewma(dataframe)

    return dataframe

In [37]:
everything = adding_boll_kelt_ewma_dataframe(df)

In [38]:
# Looking for missing values
everything.isnull().sum().sum()

61

In [39]:
# Filling missing values with the previous ones
everything = df.dropna()


In [40]:
response_data = lib_copy.signals_generator(everything)

response_data.index = response_data.index.date
final_df = response_data

In [41]:
X = final_df.iloc[:, 0:22].values
y = final_df.iloc[:, :1].values

X, y = np.array(X), np.array(y).reshape(-1,1)

In [37]:
beautiful_csv = pd.read_csv("Resources/ts_emax_x_squeeze.csv", parse_dates=[['Date', 'Time']])
beautiful_csv.set_index("Date_Time", inplace=True)
df = beautiful_csv[["squeeze", "emax_signal"]]
df.head()

,squeeze,emax_signal
Date_Time,,
2018-01-12 08:45:00,0,0
2018-01-12 09:00:00,0,0
2018-01-12 09:15:00,0,0
2018-01-12 09:30:00,0,0
2018-01-12 09:45:00,0,0


In [38]:
def target_generator(dataframe_name, col_name1, col_name2, target_col_name):
    """Creates a target for long position
    Args:
        dataframe_name (dict): Dataframe containing indicator data (0's and 1's)
        col_name1 (str): Name of first column name in dataframe to use for calculation
        col_name2 (str): Name of second column name in dataframe to use for calculation
        target_col_name (str): Name of target column name to create and store target values
    Returns:
        A dataframe of:
            original data passed to function,
            appended target column signals of type float (2.0, 1.0, 0.0)
    """
    
    # Target generation
    for index, row in dataframe_name.iterrows():
        dataframe_name.loc[index, target_col_name] = row[col_name1] + row[col_name2]

    # Return dataframe with features and target
    return dataframe_name

In [39]:
new_df = target_generator(df, "squeeze", "emax_signal", "target")
new_df.head()

,squeeze,emax_signal,target
Date_Time,,,
2018-01-12 08:45:00,0,0,0.0
2018-01-12 09:00:00,0,0,0.0
2018-01-12 09:15:00,0,0,0.0
2018-01-12 09:30:00,0,0,0.0
2018-01-12 09:45:00,0,0,0.0


In [81]:
X = new_df.iloc[:, 0:num_feature_cols].values

y = new_df.iloc[:,-1].values

X, y = np.array(X), np.array(y).reshape(-1,1)

NameError: name 'num_feature_cols' is not defined

In [41]:
print (f"X sample values:\n{X[:3]} \n")
print (f"X sample values:\n{y[:3]} \n")


X sample values:
[[3.42341000e+02 3.42341000e+02 3.41410000e+02 3.41410000e+02
  4.23600000e+03 3.39281450e+02 3.15330689e+00 3.45588064e+02
  3.32974836e+02 3.42120000e+02 9.31000000e-01 1.69407581e+00
  1.69407581e+00 3.39281450e+02 3.40975526e+02 3.37587374e+02
  3.41297157e+02 3.39800440e+02 0.00000000e+00 0.00000000e+00]
 [3.41120000e+02 3.41460000e+02 3.40610000e+02 3.40840000e+02
  6.32100000e+03 3.39533450e+02 3.06046133e+00 3.45654373e+02
  3.33412527e+02 3.41410000e+02 8.50000000e-01 1.61606473e+00
  1.61606473e+00 3.39533450e+02 3.41149515e+02 3.37917385e+02
  3.41205726e+02 3.39894945e+02 0.00000000e+00 0.00000000e+00]
 [3.40720000e+02 3.41110000e+02 3.39310000e+02 3.40790000e+02
  1.48180000e+04 3.39861450e+02 2.80284351e+00 3.45467137e+02
  3.34255763e+02 3.40840000e+02 1.80000000e+00 1.63271483e+00
  1.63271483e+00 3.39861450e+02 3.41494165e+02 3.38228735e+02
  3.41122581e+02 3.39976314e+02 0.00000000e+00 0.00000000e+00]] 

X sample values:
[[342.341]
 [341.12 ]
 [340.72

In [42]:
 # Import the StandardScaler from sklearn
from sklearn.preprocessing import StandardScaler

# Scale the data
scaler = StandardScaler().fit(X)
X = scaler.transform(X)

In [43]:
# Creating training, validation, and testing sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, random_state=78)

In [44]:
# Importing required Keras modules
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

In [45]:
# Model set-up
number_input_features = 22
hidden_nodes_layer1 = 15
hidden_nodes_layer2 = 5 
hidden_nodes_layer3 = 2

In [46]:
# Define the LSTM RNN model
model = Sequential()

# Layer 1
model.add(
    Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Layer 2
model.add(Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
model.add(Dense(1, activation="sigmoid")) 

In [47]:
# Compile the model
import tensorflow as tf

model.compile(
    loss="binary_crossentropy",
    optimizer="adam",
    metrics=[
        "accuracy",
        tf.keras.metrics.TruePositives(name="tp"),
        tf.keras.metrics.TrueNegatives(name="tn"),
        tf.keras.metrics.FalsePositives(name="fp"),
        tf.keras.metrics.FalseNegatives(name="fn"),
        tf.keras.metrics.Precision(name="precision"),
        tf.keras.metrics.Recall(name="recall"),
        tf.keras.metrics.AUC(name="auc"),
    ],
)

In [48]:
model.summary() 

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 15)                345       
_________________________________________________________________
dense_4 (Dense)              (None, 5)                 80        
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 6         
Total params: 431
Trainable params: 431
Non-trainable params: 0
_________________________________________________________________


In [49]:
# Training the model
batch_size = 1000
epochs = 50
training_history = model.fit(
    X_train,
    y_train,
    validation_data=(X_val, y_val),
    epochs=epochs,
    batch_size=batch_size,
    verbose=1,
)

Epoch 1/50


ValueError: in user code:

    /Users/markmurdock/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:806 train_function  *
        return step_function(self, iterator)
    /Users/markmurdock/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:796 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /Users/markmurdock/anaconda3/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /Users/markmurdock/anaconda3/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /Users/markmurdock/anaconda3/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    /Users/markmurdock/anaconda3/lib/p

In [50]:
 # Plotting loss
loss_df = pd.DataFrame(
    {
        "Epoch": range(1, epochs + 1),
        "Train": training_history.history["loss"],
        "Val": training_history.history["val_loss"],
    }
)
loss_df.set_index("Epoch", inplace=True)
loss_df.plot(title="Loss")

NameError: name 'training_history' is not defined

In [51]:
 # Plotting accuracy
accuracy_df = pd.DataFrame(
    {
        "Epoch": range(1, epochs + 1),
        "Train": training_history.history["accuracy"],
        "Val": training_history.history["val_accuracy"],
    }
)
accuracy_df.set_index("Epoch", inplace=True)
accuracy_df.plot(title="Accuracy")

NameError: name 'training_history' is not defined

In [52]:
# Plotting AUC
auc_df = pd.DataFrame(
    {
        "Epoch": range(1, epochs + 1),
        "Train": training_history.history["auc"],
        "Val": training_history.history["val_auc"],
    }
)
auc_df.set_index("Epoch", inplace=True)
auc_df.plot(title="AUC") 

NameError: name 'training_history' is not defined

In [53]:
# Import the roc_curve and auc metrics from sklearn
from sklearn.metrics import roc_curve, auc 

In [54]:
# Making predictions to feed the roc_curve module
train_predictions = model.predict(X_train, batch_size=1000)
test_predictions = model.predict(X_test, batch_size=1000) 

ValueError: in user code:

    /Users/markmurdock/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1462 predict_function  *
        return step_function(self, iterator)
    /Users/markmurdock/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1452 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /Users/markmurdock/anaconda3/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /Users/markmurdock/anaconda3/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /Users/markmurdock/anaconda3/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    /Users/markmurdock/anaconda3/l

In [55]:
 # Evaluate the model
scores = model.evaluate(X_test, y_test, verbose=0)

# Define metrics dictionary
metrics = {k: v for k, v in zip(model.metrics_names, scores)}

# Display evaluation metrics results
display(metrics)

ValueError: in user code:

    /Users/markmurdock/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1224 test_function  *
        return step_function(self, iterator)
    /Users/markmurdock/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1215 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /Users/markmurdock/anaconda3/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /Users/markmurdock/anaconda3/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /Users/markmurdock/anaconda3/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    /Users/markmurdock/anaconda3/lib/

In [56]:
# x_var_list = ["open","high","low","volume","bollinger_band_middle","bollinger_band_std","bollinger_band_upper","bollinger_band_lower","close_-1_s","tr","tr_14_smma","atr","kcmid","kcup","kclo","EMA9","EMA21","squeeze","crossup","crossdown","target","io_target"]

In [57]:
"""
# Construct training start and training end dates

training_start = pd.to_datetime("2018-01-12").date()
training_end = pd.to_datetime('2019-01-11').date()

# Construct test start and test end dates

testing_start = pd.to_datetime('2019-01-12').date()
testing_end = pd.to_datetime('2019-06-12').date()

# Construct validating start and validating end dates

vali_start = pd.to_datetime('2019-06-13').date()
vali_end = pd.to_datetime('2020-01-12').date()

# Confirming training, testing and validating dates
print(f"Training Start: {training_start}")
print(f"Training End: {training_end}")
print(f"Testing Start: {testing_start}")
print(f"Testing End: {testing_end}")
print(f"validating Start: {vali_start}")
print(f"validating end: {vali_end}")

"""

'\n# Construct training start and training end dates\n\ntraining_start = pd.to_datetime("2018-01-12").date()\ntraining_end = pd.to_datetime(\'2019-01-11\').date()\n\n# Construct test start and test end dates\n\ntesting_start = pd.to_datetime(\'2019-01-12\').date()\ntesting_end = pd.to_datetime(\'2019-06-12\').date()\n\n# Construct validating start and validating end dates\n\nvali_start = pd.to_datetime(\'2019-06-13\').date()\nvali_end = pd.to_datetime(\'2020-01-12\').date()\n\n# Confirming training, testing and validating dates\nprint(f"Training Start: {training_start}")\nprint(f"Training End: {training_end}")\nprint(f"Testing Start: {testing_start}")\nprint(f"Testing End: {testing_end}")\nprint(f"validating Start: {vali_start}")\nprint(f"validating end: {vali_end}")\n\n'

In [58]:
# Construct the X_train and y_train datasets
"""
X_train = final_df[x_var_list][training_start:training_end]
y_train = final_df['close'][training_start:training_end]
X_test = final_df[x_var_list][testing_start:testing_end]
y_test = final_df['close'][testing_start:testing_end]
y_test, y_train = np.array(y_test).reshape(-1,1), np.array(y_train).reshape(-1,1)
X_test, x_train = np.array(X_test), np.array(X_train).reshape(-1,1)
"""

"\nX_train = final_df[x_var_list][training_start:training_end]\ny_train = final_df['close'][training_start:training_end]\nX_test = final_df[x_var_list][testing_start:testing_end]\ny_test = final_df['close'][testing_start:testing_end]\ny_test, y_train = np.array(y_test).reshape(-1,1), np.array(y_train).reshape(-1,1)\nX_test, x_train = np.array(X_test), np.array(X_train).reshape(-1,1)\n"

In [59]:
"""
# Importing the MinMaxScaler from sklearn
from sklearn.preprocessing import MinMaxScaler

# Create a MinMaxScaler object
scaler = MinMaxScaler()

# Fit the MinMaxScaler object with the features data X
scaler.fit(X)

# Scale the features training and testing sets
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

# Fit the MinMaxScaler object with the target data Y
scaler.fit(y)

# Scale the target training and testing sets
y_train = scaler.transform(y_train)
y_test = scaler.transform(y_test)
"""

'\n# Importing the MinMaxScaler from sklearn\nfrom sklearn.preprocessing import MinMaxScaler\n\n# Create a MinMaxScaler object\nscaler = MinMaxScaler()\n\n# Fit the MinMaxScaler object with the features data X\nscaler.fit(X)\n\n# Scale the features training and testing sets\nX_train = scaler.transform(X_train)\nX_test = scaler.transform(X_test)\n\n# Fit the MinMaxScaler object with the target data Y\nscaler.fit(y)\n\n# Scale the target training and testing sets\ny_train = scaler.transform(y_train)\ny_test = scaler.transform(y_test)\n'

In [60]:
"""
# Reshape the features data
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

# Print some sample data after reshaping the datasets
print (f"X_train sample values:\n{X_train[:3]} \n")
print (f"X_test sample values:\n{X_test[:3]}")
"""

'\n# Reshape the features data\nX_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))\nX_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))\n\n# Print some sample data after reshaping the datasets\nprint (f"X_train sample values:\n{X_train[:3]} \n")\nprint (f"X_test sample values:\n{X_test[:3]}")\n'

In [61]:
"""
# Define the LSTM RNN model.
model = Sequential()

# Initial model setup
number_units = 30
dropout_fraction = 0.2

# Layer 1
model.add(LSTM(
    units=hidden_nodes_layer1,
    return_sequences=True,
    input_shape=(X_train.shape[1], 1))
    )
model.add(Dropout(dropout_fraction))

# Layer 2
model.add(LSTM(units=hidden_nodes_layer2, return_sequences=True))
model.add(Dropout(dropout_fraction))

# Layer 3
model.add(LSTM(units=hidden_nodes_layer3))
model.add(Dropout(dropout_fraction))

# Output layer
model.add(Dense(1))
"""

'\n# Define the LSTM RNN model.\nmodel = Sequential()\n\n# Initial model setup\nnumber_units = 30\ndropout_fraction = 0.2\n\n# Layer 1\nmodel.add(LSTM(\n    units=hidden_nodes_layer1,\n    return_sequences=True,\n    input_shape=(X_train.shape[1], 1))\n    )\nmodel.add(Dropout(dropout_fraction))\n\n# Layer 2\nmodel.add(LSTM(units=hidden_nodes_layer2, return_sequences=True))\nmodel.add(Dropout(dropout_fraction))\n\n# Layer 3\nmodel.add(LSTM(units=hidden_nodes_layer3))\nmodel.add(Dropout(dropout_fraction))\n\n# Output layer\nmodel.add(Dense(1))\n'

In [62]:
# Initial imports
import numpy as np
import pandas as pd
from stockstats import StockDataFrame as sdf
import requests
from dotenv import load_dotenv
import os
import pandas as pd
import numpy as np
import alpaca_trade_api as tradeapi
from pathlib import Path
import lib_copy
%matplotlib inline

In [63]:
load_dotenv()

Python-dotenv could not parse statement starting at line 7
Python-dotenv could not parse statement starting at line 8


True

In [64]:
df = lib_copy.fetch_ohlcv("TSLA", "2018-01-12", "2019-06-12")

In [65]:
def adding_boll_kelt_ewma_dataframe(dataframe):
    lib_copy.bollinger_band_generator(dataframe)
    lib_copy.keltner_channel(dataframe)
    lib_copy.ewma(dataframe)

    return dataframe

In [66]:
everything = adding_boll_kelt_ewma_dataframe(df)

In [67]:
# Looking for missing values
everything.isnull().sum().sum()

61

In [68]:
# Filling missing values with the previous ones
everything = df.dropna()

In [69]:
response_data = lib_copy.signals_generator(everything)

response_data.index = response_data.index.date
final_df = response_data

In [70]:
X = final_df.iloc[:, 0:20].values
y = final_df.iloc[:, :1].values

X, y = np.array(X), np.array(y).reshape(-1,1)

In [71]:
# Creating training, validation, and testing sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, random_state=78)

X_train, y_train = np.array(X_train), np.array(y_train).reshape(-1,1)

In [72]:
# Importing required Keras modules
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

In [73]:
# Define the LSTM RNN model.
model = Sequential()

# Initial model setup
number_units = 30
dropout_fraction = 0.2

# Layer 1
model.add(LSTM(
    units=number_units,
    return_sequences=True,
    input_shape=(X_train.shape[1], 1))
    )
model.add(Dropout(dropout_fraction))

# Layer 2
model.add(LSTM(units=number_units, return_sequences=True))
model.add(Dropout(dropout_fraction))

# Layer 3
model.add(LSTM(units=number_units))
model.add(Dropout(dropout_fraction))

# Output layer
model.add(Dense(1))

In [74]:
# Compile the model
model.compile(optimizer="adam", loss="mean_squared_error")

In [75]:
# Show the model summary
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_9 (LSTM)                (None, 20, 30)            3840      
_________________________________________________________________
dropout_9 (Dropout)          (None, 20, 30)            0         
_________________________________________________________________
lstm_10 (LSTM)               (None, 20, 30)            7320      
_________________________________________________________________
dropout_10 (Dropout)         (None, 20, 30)            0         
_________________________________________________________________
lstm_11 (LSTM)               (None, 30)                7320      
_________________________________________________________________
dropout_11 (Dropout)         (None, 30)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 1)                

In [76]:
batch_size = 1000
epochs = 10
training_history = model.fit(X_train, y_train, epochs=epochs, shuffle=False, batch_size=batch_size, validation_data=(X_val,y_val),verbose=1)

Epoch 1/10


ValueError: in user code:

    /Users/markmurdock/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:806 train_function  *
        return step_function(self, iterator)
    /Users/markmurdock/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:796 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /Users/markmurdock/anaconda3/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /Users/markmurdock/anaconda3/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /Users/markmurdock/anaconda3/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    /Users/markmurdock/anaconda3/lib/p

In [77]:
# Evaluate the model
model.evaluate(X_test, y_test, verbose=0)

ValueError: in user code:

    /Users/markmurdock/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1224 test_function  *
        return step_function(self, iterator)
    /Users/markmurdock/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1215 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /Users/markmurdock/anaconda3/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /Users/markmurdock/anaconda3/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /Users/markmurdock/anaconda3/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    /Users/markmurdock/anaconda3/lib/

In [78]:
# Make predictions using the testing data X_test
predicted = model.predict(X_test)

ValueError: in user code:

    /Users/markmurdock/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1462 predict_function  *
        return step_function(self, iterator)
    /Users/markmurdock/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1452 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /Users/markmurdock/anaconda3/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /Users/markmurdock/anaconda3/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /Users/markmurdock/anaconda3/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    /Users/markmurdock/anaconda3/l

In [79]:
# Recover the original prices instead of the scaled version
predicted_prices = scaler.inverse_transform(predicted)
real_prices = scaler.inverse_transform(y_test.reshape(-1, 1))

NameError: name 'predicted' is not defined

In [80]:
# Create a DataFrame of Real and Predicted values
stocks = pd.DataFrame({
    "Actual": real_prices.ravel(),
    "Predicted": predicted_prices.ravel()
}, index = final_df.index[-len(real_prices): ]) 

# Show the DataFrame's head
stocks.head()

NameError: name 'real_prices' is not defined